In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string

2023-11-23 21:51:04.503253: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-23 21:51:04.537854: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 21:51:04.537880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 21:51:04.538692: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-23 21:51:04.543758: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-23 21:51:04.544312: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
df=pd.read_csv('one_output_dataset.csv')
df

,speech,emotion
0,From the moment that the French defenses at Se...,anger
1,We observe today not a victory of party but a ...,joy
2,"Your Majesties, Your Highnesses, Distinguished...",trust
3,I am honored to be with you today at your comm...,anticipation
4,"Honorable UN Secretary General Mr Ban Ki-moon,...",sadness
5,It is with a profound sense of humility that I...,sadness
6,My message is that we'll be watching you.\r\n\...,fear
7,"Hello everybody. You know, Michelle and I have...",joy
8,"Your Majesties, Your Royal Highness, Excellenc...",anticipation
9,"Five score years ago, a great American, in who...",trust


In [3]:
#preprocess transcript

# download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# initialize a PorterStemmer
stemmer = PorterStemmer()

def preprocess_text(text):
    # lowercase the text
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # tokenize the text
    words = word_tokenize(text)
    # remove stopwords and stem the words
    words = [stemmer.stem(word) for word in words if word not in stopwords.words('english')]
    # join the words back into a string
    text = ' '.join(words)

    return text

df['speech'] = df['speech'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /home/arjun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/arjun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
tokenizer=AlbertTokenizer.from_pretrained('albert-large-v2')

model=TFAlbertForSequenceClassification.from_pretrained('albert-large-v2',num_labels=6)

2023-11-23 21:51:21.701741: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-11-23 21:51:21.701775: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: fernblade
2023-11-23 21:51:21.701783: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: fernblade
2023-11-23 21:51:21.701903: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 545.29.2
2023-11-23 21:51:21.701927: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 545.29.2
2023-11-23 21:51:21.701934: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 545.29.2
All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceCla

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [7]:
#split in train and test in text and labels
train_texts = train['speech'].tolist()
train_labels = train['emotion'].tolist()

test_texts = test['speech'].tolist()
test_labels = test['emotion'].tolist()

In [8]:
#tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

In [9]:
#convert features and labels to tensors for both train and test
train_features = {key: tf.convert_to_tensor(val) for key, val in train_encodings.items()}
train_labels = tf.convert_to_tensor(train_labels)

test_features = {key: tf.convert_to_tensor(val) for key, val in test_encodings.items()}
test_labels = tf.convert_to_tensor(test_labels)

In [10]:
#prepare the training and testing dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
train_dataset = train_dataset.shuffle(10000).batch(2)

test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels))
test_dataset = test_dataset.batch(2)

In [11]:
#define model metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

In [12]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [13]:
model.summary()

Model: "tf_albert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 albert (TFAlbertMainLayer)  multiple                  17683968  
                                                                 
 dropout_4 (Dropout)         multiple                  0         
                                                                 
 classifier (Dense)          multiple                  6150      
                                                                 
Total params: 17690118 (67.48 MB)
Trainable params: 17690118 (67.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
#fit model to train dataset
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3


2023-11-23 21:59:11.845496: W tensorflow/core/framework/op_kernel.cc:1816] OP_REQUIRES failed at cast_op.cc:122 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node Cast_1 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/h